In [1]:
import os
import time
import Turbine
import mat_properties as prop
import numpy as n
import pandas as pd

os.environ["RPPREFIX"] = r"C:/Program Files (x86)/REFPROP"

RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")

water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    RP=RP,
)


# таблица номинального режима
gas_streams0 = pd.read_excel("streams0.xlsx", sheet_name="gas", index_col=0)
water_streams0 = pd.read_excel("streams0.xlsx", sheet_name="water", index_col=0)
# таблица задаваемых значений (заменится на интерфейс)
gas_streams = pd.read_excel("streams.xlsx", sheet_name="gas", index_col=0)
water_streams = pd.read_excel("streams.xlsx", sheet_name="water", index_col=0)

water_streams['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S,X
DROSVD-TURBVD,492.20,5.1990,3414.093682,41.97,NaN,NaN
ENDOFVD,210.00,0.5490,2874.837567,41.97,NaN,NaN
DROSND-TURBND,199.40,0.4000,2859.669112,12.88,NaN,NaN
SMESHEND,78.16,0.4000,327.573430,54.85,NaN,NaN
DOOTB2,131.40,0.2542,2724.861533,54.85,NaN,NaN
DOOTB1,NaN,0.1525,2686.016192,54.85,NaN,NaN
INCND,NaN,0.1525,2686.016192,54.85,NaN,NaN
INKOND,NaN,0.0017,2341.001493,54.85,NaN,NaN


In [6]:
calctolerance = 10**-3
calcmethod = "hybr"
KPD_SP = 0.99  # эффективность сетевых подогревателей
RP = prop.init_REFPROP(r"C:\Program Files (x86)\REFPROP")
water = prop.Materials_prop(
    "water",
    [1.0, 0, 0, 0, 0],
    prop.REFPROP_h_s,
    prop.REFPROP_p_t,
    prop.REFPROP_p_h,
    prop.REFPROP_p_s,
    prop.REFPROP_p_q,
    prop.REFPROP_t_q,
    RP=RP,
)

Turb = Turbine.turbine(
    water,
    "DROSVD-TURBVD",
    "ENDOFVD",
    "DROSND-TURBND",
    "SMESHEND",
    "DOOTB2",
    "DOOTB1",
    "INCND",
    "INKOND",
    water_streams0,
    water_streams,
)

start_time = time.time()

res = Turb.calculate(diafragma=0.1, maxiterations=50, calctolerance=10**-3)

print("fin:--- %s сек. ---" % round((time.time() - start_time), 1))
res

Максимальная погрешность определения давления в отборах 0.0005343233926672484
fin:--- 0.1 сек. ---


{'Max_error': 0.0005343233926672484,
 'P_out': [5.19262954031339,
  0.4579057458202341,
  0.5199057458202341,
  0.4579057458202341,
  0.30019038131584774,
  0.21155525200205377,
  0.11155525200205378,
  0.0017],
 'Eff_out': [0.8493807611371985,
  0.8372415159994411,
  0.8149716606831633,
  0.7339885635728762]}

In [7]:
# надо посчитать потерю давления НД (решаю)
# отборы неправильно
print(water_streams['DROSVD-TURBVD':'INKOND'])

                        T         P            H      G         S           X
DROSVD-TURBVD  492.167815  5.192630  3414.093682  41.97  6.934615  100.000000
ENDOFVD        208.449850  0.457906  2875.870557  41.97  7.142575  100.000000
DROSND-TURBND  202.261225  0.519906  2859.669112  12.88  7.051568  100.000000
SMESHEND       206.646478  0.457906  2872.066097  54.85  7.134661  100.000000
DOOTB2         167.893936  0.300190  2799.263343  54.85  7.166996  100.000000
DOOTB1         138.779301  0.211555  2744.754459  54.85  7.197253  100.000000
INCND          134.506113  0.111555  2744.754459  54.85  7.488090  100.000000
INKOND          14.947194  0.001700  2312.330367  54.85  8.032068   91.242806


In [4]:
water_streams0['DROSVD-TURBVD':'INKOND']

,T,P,H,G,S
DROSVD-TURBVD,508.600000,7.8450,3422.525472,63.40,NaN
ENDOFVD,195.852578,0.5490,2844.285130,63.40,NaN
DROSND-TURBND,209.800000,0.6110,2871.426945,14.76,7.003739
SMESHEND,197.800000,0.5490,2848.521603,78.16,7.003601
DOOTB2,131.400000,0.2542,2724.861533,78.16,7.065770
DOOTB1,NaN,0.1525,2652.000000,78.16,7.109000
INCND,NaN,0.1525,2652.000000,78.16,NaN
INKOND,NaN,0.0080,2338.024560,78.16,NaN
